### Funzioni variadiche
*args è una tupla, **kwargs un dizionario (bisogna passare le keyword)

In [2]:
def func(*args,**kwargs):
    print(args)
    print(kwargs)
func(2,4,5,var1=3,var2 =3)

(2, 4, 5)
{'var1': 3, 'var2': 3}


# Classi

Composizione vs inheritance: Composizione (un' attributo della classe è istanza di un'altra classe) = relazione di possesso. Inheritance = relazione di identità

### Variabili "private"

In python non esistono variabili private ma esistono delle convenzioni per far si che sia più difficile per l'utente rompere accidentalmente la classe facendo assegnazioni sbagliate alle variabili della classe.

1. Usare _ (underscore) come prefisso dell'attributo
2. creare un metodo che ritorna l'attributo e decorarlo con **property** (questo fa in modo che il metodo sia chiamabile senza usare l'operatore di chiamata () )
3. (Opzionale): definisci un setter method per consentire la modifica dell'attributo (volendo si possono inserire dei meccaniscmi di controllo). va decorata con **attribute_name-setter**

Nota che si potrà comunque accedere a _attr_name

In [26]:
class Particle():
    def __init__(self, momentum):
        self._momentum = momentum

    @property
    def momentum(self):
        return self._momentum

    @momentum.setter
    def momentum(self, value):
        if (value>=0):
            self._momentum = value
        else:
            print("Momentum must be positive")
    def class_name(self):
        print("Particle")
mu=Particle(10)
print(mu.momentum)
mu.momentum=5
print(mu.momentum)
mu.momentum=-4
print(mu.momentum)
mu._momentum=-4 ##si puo comunque accedere a questo attributo
print(mu.momentum)

10
5
Momentum must be positive
5
-4


In [29]:
#INHERITANCE (i metodi ereditati possono essere sovrascritti)
class ChargedParticle (Particle):
    def __init__(self, momentum, charge):
        Particle.__init__(self,momentum)
        self.charge = charge
mu=ChargedParticle(10,-1)
mu.momentum #Nota che ha ereditato anche la property

10

### Dunder methods

E' possibile vedere tutti gli attributi di una classe tramite la funzione **dir()**

- \_\_abs__,  \_\_add\_\_ , \_\_len\_\_,\_\_sub\_\_,\_\_mul\_\_,\_\_truediv\_\_,\_\_floordiv\_\_,\_\_pow\_\_ ... banali

- \_\_rmul\_\_  gestisce la moltiplicazione ma a destra (facendo a*b l'interprete cerca il \_\_mul\_\_ di a, se non c'è cerca \_\_rmul\_\_ di b)

- \_\_iadd\_\_ ,\_\_isub\_\_,\_\_imul\_\_,\_\_itruediv\_\_,\_\_ifloordiv\_\_   modifica il comportamento di +=,-=,*=,/=,//=

- \_\_lt\_\_, \_\_le\_\_,\_\_eq\_\_,\_\_ne\_\_,\_\_gt\_\_,\_\_ge\_\_ cambia il comportamento degli operatori <,<=,==,!=,>,>=. NB: una volta definita una coppia di operatori complementari (ad esempio >= e <) python automaticamente definisce in modo coerente tutti gli altri operatori. Inoltre una volta aver definito le relazioni di equivalenza è possibile usare il metodo .sort() sull'oggetto da noi creato

- \_\_str\_\_ modifica il comportamtento di print()

- \_\_repr\_\_ torna un output più dettagliato per il debug (Se non esiste un str l'interprete cerca direttamente repr)

- \_\_contains\_\_ modifica il comportamento di in

- \_\_setitem\_\_ è il metodo chiamato quando si fa un assegnazione a un dizionario. E' possibile dotare una classe della stessa funzionalità, cioè istance['key']=value dopo aver definito def \_\_setitem\_\_(self,name,value)

- \_\_getitem\_\_ come setitem ma si usa per richiamare il valore di una chiave (ovviamente solo def \_\_getitem\_\_(self,name))

- \_\_delitem\_\_ modifica il comportamento di rimozione della chiave

- \_\_missing\_\_ cambia il comportamente in caso di chiave mancante del dizionario

- \_\_del\_\_ modifica il comportamento di del ( non esattamente così, fa prima altre cose)

- \_\_call\_\_ metodo chiamato ogni volta che viene invocata una funzione (o meglio, overloading dell'operatore call () ). Può essere usato per assegnare alla classe il comportamento di funzione 

  ```python
  class name:
      def __call__(self):
          return True
  istance=name()
  istance() #torna True
  ```

  Questo mostra come **le funzioni sono classi**

- \_\_hash__ è possibile definire l'hashing su un oggetto da noi definito.
  Per poter definire un hashing serve che
  - L'oggetto sia immutabile
  - Sia dotato di metodo \_\_eq\_\_ per comparare gli oggetti della classe
  - deve tornare lo stesso hash per oggetti equivalenti, tornare lo stesso valore per oggetti diversi e ricoprire il codominio in modo uniforme

La lista di tutti i dunder method è qui https://docs.python.org/3/reference/datamodel.html#

## Metaclassi
Una metaclasse è una class factory. Quando viene creato un oggetto viene chiamata la metaclasse della classe a cui appartiene l'oggetto

Una metaclasse eredita da type, deve essere dotata di un metodo call che prende in input cls e ritorna una istanza (usualmente creata usando il call di type che è la metaclasse predefinita).

Per usare la metaclasse la classe andrà definita con (metaclass=metaclass_name)

Per esempio possono essere usate per creare un singleton

In [97]:
class MetaSingleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs) ## Chiama il costruttore della classe usando la metaclasse type (quella di default)
        return cls._instances[cls]
class Singleton(metaclass=MetaSingleton):
    def __init__(self,data):
        self.data=data
    pass

sing=Singleton(2)
sing2=Singleton(3)
print(sing2.data) # sing2 in realtà è sing

2


2

# Iterabili e iteratori
Un iterabile è un oggetto che ha il metodo \_\_iter\_\_ che ritorna un iteratore.
Un iteratore è un oggetto che implementa il medoto \_\_next\_\_ che dice quale sia l'elemento successivo nella sequenza
Quando non ci sono più elementi da ritornare viene raisata un'eccezione StopIteration

Grazie alla presenza del metodo \_\_iter\_\_ è possibile usare tutti i metodi applicati alle iterabili come :

- sum(), max(), min(), enumerate
- map(): Applica una funzione elemento per elemento ad un iterabile
- filter(): Itera solamente  su gli elementi che rispettano una data condizione
- zip(): Itera su una coppia di elementi ovvero crea coppie di elementi di due iterabili

Inoltre la libreria **itertools** presenta una serie sterminata di strumenti da usare sugli iterabili:

- takewhile: Stoppa il looping quando una condizione diventa falsa
- chain: Looppa su più sequenza una dopo l'altra
- cycle: Looppa sulle sequenze in modo ripetuto indefinitamente
- permutations: Crea su tutte le permutazioni di elementi di una data lungheza

## Generatori

I generatori sono oggetti che funzionano in modo Lazy: generano i dati solo al momento di iterazione evitando l'overhead in memoria. (Un esempio è range)
Per creare un generatore basta usare **yield**

Le funzioni map, zip, filter, reverse, enumerate ritornano dei generatori

In [54]:
#esempio iteratore (esempio brutale, non è ovviamente il modo migliore per fare questa cosa)
class MinkowskiVec1D():
    def __init__(self, x,t):
        self.x=x
        self.t=t
        self.MVec=list(zip(t,x))
        self.index=0
    def __next__(self):
        try:
            item=self.MVec[self.index]
        except IndexError:
            raise StopIteration
        self.index +=1
        return list(item)
    def __iter__(self):
        return self
    def generator(self):
        for elem in self.MVec:
            yield elem

vec1=MinkowskiVec1D([1,2,3,4,5,6,7,8,9,10], [2,3,4,5,6,7,8,9,10,11])
print(vec1.MVec)
for i in vec1:
    print(i)
vec_generator=vec1.generator()
print(vec_generator)
for i in vec_generator:
    print(i)



[(2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (7, 6), (8, 7), (9, 8), (10, 9), (11, 10)]
[2, 1]
[3, 2]
[4, 3]
[5, 4]
[6, 5]
[7, 6]
[8, 7]
[9, 8]
[10, 9]
[11, 10]
<generator object MinkowskiVec1D.generator at 0x7fc4a68ff370>
(2, 1)
(3, 2)
(4, 3)
(5, 4)
(6, 5)
(7, 6)
(8, 7)
(9, 8)
(10, 9)
(11, 10)


# Decoratori

Un decoratore è un modo per poter aggiungere delle funzionalità a una funzione wrappandola all'interno di un'altra funzione.

Una funzione è anche decorabile più volte, basta impilare i decoratori. La funzione verrà comunque eseguita una sola volta

Per creare un decoratore si fa prima a mostrare un esempio che a spiegarlo a parole ma essenzialmente chiamare una funzione decorata equivale a chiamare la funzione decorator_name(function_name) (Si può vedere il decoratore come funzione di funzione)

**Problema** In questo modo perdo sia le docstring che alcune informazioni come \_\_name\_\_ della funzione decorata (function.\_\_name\_\_ sarà "wrapper"). Per risolvere questo problema posso usare il decoratore wraps da functools e decorare il wrapper

In [72]:
# Decoratore per mostrare il tempo impiegato per eseguire una funzione
import time
from functools import wraps

def elapsed_time(func):
    @wraps(func)
    def wrapper(*args,**kwargs):
        print(f"I'm about to call '{func.__name__}'")
        start=time.time()
        ret=func(*args,**kwargs)
        print("Elapsed time:",time.time()-start)
        return ret
    return wrapper

@elapsed_time
def funzione():
    print("Hello")
    time.sleep(1)
    return 1

ret=funzione()
print(ret)
funzione.__name__


I'm about to call 'funzione'
Hello
Elapsed time: 1.0010275840759277
1


'funzione'

Volendo si può definire anche un **decoratore con dei parametri**, basta incapsulare il decoratore in un' altra funzione che ha in input i parametri e che ritorna il decoratore

In [75]:
def elapsed_attr(param):
    print(f"Arguments:{param}")
    return elapsed_time

@elapsed_attr(2)
def funzione():
    print("Hello")
    time.sleep(1)
    return 1

ret=funzione()

Arguments:2
I'm about to call 'funzione'
Hello
Elapsed time: 1.0010526180267334


## classmethods
Un classmethod è un metodo appartenente a una classe che si può usare senza definire un instanza. E' utile per definire un costruttore alternativo

**NB: non essendo riferito a nessuna istanza non può accedere ai suoi attributi e non ha self tra gli attributi (ma si può passare cls che è riferito all'intera classe)**

In [86]:
import numpy as np
class Data:
    def __init__(self, data):
        self.data = data

    @classmethod
    def fromFile(cls, filename):
        try:
            data = np.loadtxt(filename)
            return cls(data)
        except:
            print(f"File {filename} not found")
data1=Data([1,2,3])
print(data1.data)
data2=Data.fromFile("data.txt")

[1, 2, 3]
File data.txt not found


## staticmethod
Uno staticmethod è un metodo che non riceve l'istanza o la classe come primo argomento e non altera lo stato della classe. E' definito nella classe per pura convenienza ma potrebbe stare tranquillamente fuori

In [88]:
class angle:
    @staticmethod
    def rad2deg(rad):
        return rad*180/np.pi
deg_pi=angle.rad2deg(np.pi)
print(deg_pi)

180.0


# Eccezioni
E' possibile definire un eccezione custom creando una classe che eredita da un'eccezione

In [109]:

class ValoreBrutto(ValueError):
    def __init__(self, value):
        self.value = value
        ValueError.__init__(self,f"Bro, ma come ti viene in mente di scegliere {self.value}?")

raise ValoreBrutto(10)

ValoreBrutto: Bro, ma come ti viene in mente di scegliere 10?

# Contex managers

Un contex manager è una classe dotata dei metodi \_\_enter\_\_ ed \_\_exit\_\_

1. with chiama il metodo enter e il valore ritornato verrà assegnato alla variabile dopo il comando as
2. Vengono eseguiti i comandi all'interno del contex manager
3. Viene eseguito l'exit method (exit oltr a self ha bisogno di exc_type,exc_value, exc_traceback). Exit ritorna un valore booleano che indica se è stata raisata un'eccezione nel corpo di with. I 3 argomenti aggiuntivi contengono informazioni sull'eccezione 

In [1]:
import glob
import os
class filedisplay:
    def __init__ (self,path=os.getcwd()):
        self.path=path
    def __enter__(self):
        print(self.path)
        return glob.glob(f"{self.path}/*")
    def __exit__(self,exc_type,exc_value, exc_traceback):
        print("Contex manager exited")
with filedisplay() as files:
    print(f"The number of files in this directory is {len(files)}")
    print(files)


/home/pviscone/Desktop/CMEPDA/python
The number of files in this directory is 1
['/home/pviscone/Desktop/CMEPDA/python/python.ipynb']
Contex manager exited
